# 2. Data handling and visualization

IN GENERAL: INTRODUCE THINGS HERE WHICH WE NEED IN SESSIONS 3-14. THAT MEANS, WHENEVER WE 

### Textbooks & sources

- https://jakevdp.github.io/PythonDataScienceHandbook/
- https://www.pythonlikeyoumeanit.com/index.html

### Notes

- Focus here is on flat data structures (Pandas dataframes) and mathematical data structures (NumPy arrays), add also regex here instead of showing them in the functions of NLP?; hierarchical data structures (JSON and HTML) are covered in session 4.

## 2.1. Essentials

https://www.pythonlikeyoumeanit.com/module_2.html

BOX: OBJECT-ORIENTED PROGRAMING
https://www.pythonlikeyoumeanit.com/module_4.html

## 2.2. Pandas

- https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html

### 2.2.1. TweetsCOV19 dataset

https://data.gesis.org/tweetscov19/

- Tweet Id: Long.
- Username: String. Encrypted for privacy issues.
- Timestamp: Format ( "EEE MMM dd HH:mm:ss Z yyyy" ).
- #Followers: Integer.
- #Friends: Integer.
- #Retweets: Integer.
- #Favorites: Integer.
- Entities: String. For each entity, we aggregated the original text, the annotated entity and the produced score from FEL library. Each entity is separated from another entity by char ";". Also, each entity is separated by char ":" in order to store "original_text:annotated_entity:score;". If FEL did not find any entities, we have stored "null;".
- Sentiment: String. SentiStrength produces a score for positive (1 to 5) and negative (-1 to -5) sentiment. We splitted these two numbers by whitespace char " ". Positive sentiment was stored first and then negative sentiment (i.e. "2 -1").
- Mentions: String. If the tweet contains mentions, we remove the char "@" and concatenate the mentions with whitespace char " ". If no mentions appear, we have stored "null;".
- Hashtags: String. If the tweet contains hashtags, we remove the char "#" and concatenate the hashtags with whitespace char " ". If no hashtags appear, we have stored "null;".
- URLs: String: If the tweet contains URLs, we concatenate the URLs using ":-: ". If no URLs appear, we have stored "null;"


Download the file https://zenodo.org/record/4593502/files/TweetsCOV19_052020.tsv.gz and store it in the data/tweetscov19/ directory.

In [1]:
import pandas as pd
import numpy as np

In [2]:
tweets = pd.read_csv('data/data', sep='\t', header=None)

In [3]:
tweets.columns = ['tweet_id', 'username', 'timestamp', 'followers', 'friends', 'retweets', 'favorites', 'entities', 'sentiment', 'mentions', 'hashtags', 'urls']

In [4]:
# tweets = tweetss.copy()

TO IMPLEMENT:
- SPLIT 'sentiment' INTO 'sentiment_pos' AND 'sentiment_neg' AND DELETE THE ORIGINAL SERIES
- CREATE AN 'entities' DF WITH FOUR COLUMNS ['tweet_id', 'original', 'annotated', 'score']
- CREATE A 'mentions' DF WITH ['tweet_id', 'mentionee'] COLUMNS
- CREATE A 'hashtags' DF WITH ['tweet_id', 'hashtag'] COLUMNS
- CREATE A 'urls' DF WITH ['tweet_id', 'url'] COLUMNS

In [5]:
# Splitting 'sentiment' into pos and neg and deleting 'sentiment' column:

pos = []
neg = []

for i in tweets['sentiment']:
    pos.append(i.split()[0])
    neg.append(i.split()[1])
    
tweets['sentiment_pos'] = pos
tweets['sentiment_neg'] = neg

del tweets['sentiment']

# tweets

In [6]:
# Putting values of hashtags column into lists:
# Note: 'null;' and NaN values are replaced with ['']

import math

hashtags = []

for i in tweets['hashtags']:
    if i == 'null;' or type(i) == float:
        hashtags.append([''])
#     elif type(i) == float:
#         hashtags.append(float('nan'))
    else:
        hashtags.append(i.split())
        
tweets['hashtags'] = hashtags




# Putting values of mentions column into lists:
# Note: 'null;' and NaN values are replaced with ['']

mentions = []

for i in tweets['mentions']:
    if i == 'null;' or type(i) == float:
        mentions.append([''])
#     elif type(i) == float:
#         mentions.append(float('nan'))
    else:
        mentions.append(i.split())
        
tweets['mentions'] = mentions




# Putting values of entities column into lists:
# Note: 'null;' values are replaced with ['']

entities = []

for i in tweets['entities']:
    
    if i == 'null;':
        entities.append([''])
    else:
        splitted = i.split(';')
        del splitted[-1]
        entities.append(splitted)
    
        
tweets['entities'] = entities




# Putting values of urls column into lists:
# Note: 'null;' and NaN values are replaced with ['']

urls = []

for i in tweets['urls']:
    if i == 'null;' or type(i) == float:
        urls.append([''])
#     elif type(i) == float:
#         mentions.append(float('nan'))
    else:
        splitted = i.split(':-:')
        del splitted[-1]
        urls.append(splitted)
        
tweets['urls'] = urls

In [9]:
# Creating users dataframe:

# Note: This may take around 90 seconds to run

followers_max = tweets.loc[tweets.groupby('username')['followers'].idxmax()]
followers_max = followers_max.reset_index()

friends_max = tweets.loc[tweets.groupby('username')['friends'].idxmax()]
friends_max = friends_max.reset_index()

users = pd.DataFrame()
users['username'] = friends_max['username']
users['followers_max'] = followers_max['followers']
users['friends_max'] = friends_max['friends']
users

,username,followers_max,friends_max
0,00000998260226834ffdbdf98ff33eb7,1852,1482
1,000016e54a4dc155432ebad949c2546e,6953,992
2,00001c34da8eab17b175a9e049078b72,341,350
3,00001d45dd97d52b5accb3333e3790e3,854,3012
4,00003291a067882da356e7f963d3dca8,104,805
...,...,...,...
1117996,ffffd2b829300cc638eb4c78c0fc1882,2987,2553
1117997,ffffd8b7f90bd8937218b42ee841dc20,366,817
1117998,ffffda7501c5f86d5ae850ca7a9fbd1f,1924,925
1117999,ffffeca2c4676546be82c9bf9df9c322,335,521


In [13]:
# Creating entities dataframe:

entities_table = pd.DataFrame()

res = pd.DataFrame({'entities': np.concatenate(tweets['entities'].values)})
entities_table['entities'] = res.squeeze().value_counts().index

original = ['']
annotated = ['']
score = ['']

for i in entities_table['entities'][1:]:      
          
    split = i.split(':')
    original.append(split[0])
    annotated.append(split[1])
    score.append(split[2])

entities_table['original'] = original
entities_table['annotated'] = annotated
entities_table['score'] = score

entities_table['selections'] = res.squeeze().value_counts().reindex().to_numpy()
    
entities_table

,entities,original,annotated,score,selections
0,,,,,559396
1,covid 19:Coronavirus_disease_2019:-1.535776454...,covid 19,Coronavirus_disease_2019,-1.535776454600282,140238
2,quarantine:Quarantine:-2.3096035868012508,quarantine,Quarantine,-2.3096035868012508,70605
3,china:China:-2.113921624336916,china,China,-2.113921624336916,57153
4,social distancing:Social_distancing:-1.4103273...,social distancing,Social_distancing,-1.4103273474020743,38290
...,...,...,...,...,...
181966,steve biko:Steve_Biko:-1.1217053124820895,steve biko,Steve_Biko,-1.1217053124820895,1
181967,jack nicklaus:Jack_Nicklaus:-0.810259216283029,jack nicklaus,Jack_Nicklaus,-0.810259216283029,1
181968,fine living:Cooking_Channel:-2.3010870202064426,fine living,Cooking_Channel,-2.3010870202064426,1
181969,3des:Triple_DES:-2.5412059066401578,3des,Triple_DES,-2.5412059066401578,1


In [12]:
# Creating mentions dataframe:

mentions_table = pd.DataFrame()

res = pd.DataFrame({'mentions': np.concatenate(tweets['mentions'].values)})
mentions_table['mentions'] = res.squeeze().value_counts().index

mentions_table['selections'] = res.squeeze().value_counts().reindex().to_numpy()

mentions_table

,mentions,selections
0,,1087388
1,realDonaldTrump,37878
2,PMOIndia,6361
3,narendramodi,6342
4,jaketapper,5892
...,...,...
678375,uniquely_q,1
678376,jmasseypoet,1
678377,isakeven95,1
678378,abbahmadAbk,1


In [11]:
# Creating hashtags dataframe:

hashtags_table = pd.DataFrame()

res = pd.DataFrame({'hashtags': np.concatenate(tweets['hashtags'].values)})
hashtags_table['hashtags'] = res.squeeze().value_counts().index

hashtags_table['selections'] = res.squeeze().value_counts().reindex().to_numpy()

hashtags_table

,hashtags,selections
0,,1346705
1,COVID19,67608
2,coronavirus,30422
3,null;,21739
4,Covid_19,11057
...,...,...
379991,10thexampostpone,1
379992,Swadesi.,1
379993,WhatsAppFakeForward,1
379994,Afghanistannews,1


In [10]:
# Creating urls dataframe:

urls_table = pd.DataFrame()

urls1 = pd.DataFrame()
urls1['urls'] = tweets['urls']

urls2 = pd.DataFrame()

for i in range (10):
    
    temp_df = urls1[i*(int(urls1.shape[0]/10)):i*(int(urls1.shape[0]/10))+int(urls1.shape[0]/10)]
    res = pd.DataFrame({'urls': np.concatenate(temp_df['urls'].values)})
    urls2 = pd.concat([urls2, res], ignore_index = True, axis = 0)



urls_table['urls'] = urls2.squeeze().value_counts().index

urls_table['selections'] = urls2.squeeze().value_counts().reindex().to_numpy()

urls_table

,urls,selections
0,,1397146
1,https://www.twittascope.com/?sign=5,549
2,https://api.whatsapp.com/send?phone=9190393567...,368
3,http://rebrand.ly/work-2020,286
4,https://www.twittascope.com/?sign=6,271
...,...,...
420620,https://mol.im/a/8350307,1
420621,https://iq.cash,1
420622,https://block.fiverr.com/index.html?url=aHR0cD...,1
420623,https://www.newsweek.com/wisconsin-woman-licks...,1


In [ ]:
# Function to convert the dataframe to the first normal form:


# def first_normal_form(maindf, column: str, sections: int):
    
#     resultdf = pd.DataFrame()

#     for i in range(sections):

#         df = maindf[i*(int(maindf.shape[0]/sections)):i*(int(maindf.shape[0]/sections))+int(maindf.shape[0]/sections)]
         
#         lens = list(map(len, df[column].values))

#         column_loc = df.columns.get_loc(column)

#         df_dict = {}
        
#         for i in range(column_loc):
#             df_dict[df.columns[i]] = np.repeat(df[df.columns[i]], lens)

#         df_dict[column] = np.concatenate(df[column].values)

#         for i in range(13 - column_loc - 1):
#             df_dict[df.columns[i+column_loc+1]] = np.repeat(df[df.columns[i+column_loc+1]], lens)
        
#         fnf = pd.DataFrame(df_dict)
        
#         resultdf = pd.concat([resultdf, fnf], ignore_index = True, axis = 0)
            
#     return resultdf

In [ ]:
# firstNF1 = first_normal_form(tweets, 'entities', 10) 

# firstNF2 = first_normal_form(firstNF1, 'hashtags', 15)
# del firstNF1

# firstNF3 = first_normal_form(firstNF2, 'mentions', 25)
# del firstNF2

# firstNF4 = first_normal_form(firstNF3, 'urls', 50)
# del firstNF3

### 2.2.2. Working with a single dataframe

In [15]:
# read/save
# describe()
# changing index and column names
# grouping
# using and resetting the index
# categorize series: categories and codes
# matrix to edgelist and vice versa
# zip
# columns into dict
# datetime
# ...

### 2.2.3. Working with multiple dataframes

In [24]:
# merge split concat etc
# ...

## 2.3. NumPy

- https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html
- https://www.pythonlikeyoumeanit.com/module_3.html

In [25]:
# read/save
# relationship to pandas
# ...

## 2.4. SciPy

In [1]:
# sparse matrices
# matrix multiplication

## 2.5. Data visualiation with Seaborn & Matplotlib

SUGGESTION: TEACH HOW TO WITH SEABORN, USE MATPLOTLIB WHERE SEABORN DOES NOT OFFER METHODS

- https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html

In [23]:
# Some drafts

# lst = list(tweets['urls'])
# print (len(lst))
# lst2 = list(filter(('null;').__ne__, lst))
# len(lst2)

In [24]:
# Some drafts

# import pandas as pd
# import numpy as np
 
# df = pd.DataFrame({'Name': ['David', 'Glenn', 'Steve'], 'Subjects': [
#                   ['English', 'Math'], ['Math'], ['Science', 'English']]})

# df2 = pd.DataFrame({'Name': ['David', 'Glenn', 'Steve'], 'Subjects': [
#                   ['English', 'Math'], ['Math'], ['Science', 'English']], 'Subjects2': [['s1', 's2','s3'], ['Math'], ['s1', 's2']]})

# lens = list(map(len, df2['Subjects'].values))
 
# res = pd.DataFrame({'Name': np.repeat(df2['Name'], lens), 'Subject': np.concatenate(df2['Subjects'].values), 'Subject2': np.repeat(df2['Subjects2'], lens)})
 
    
# df2